In [2]:
from dotenv import load_dotenv, find_dotenv
import chromadb
load_dotenv(find_dotenv())

True

In [3]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf")
pages = loader.load_and_split()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True,
)

In [5]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name='angad_embedding')

2023-10-17 19:03:25 - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [6]:
collection

Collection(name=angad_embedding)

In [7]:
texts = text_splitter.create_documents([pages[1].page_content])
texts

[Document(page_content='Abstract\nElectricity is the backbone of today’s society. It powers everything from houses to industries and\nbusinesses. The core infrastructure behind electricity is the grid which requires a continual balance\nbetween supply and demand for optimal functioning. This necessitates the accurate short-term\nforecasting of the load (STLF) consumed. In recent years, deep learning has been employed with\nfederated learning for STLF as federated learning allows collaboration of clients without the need\nto aggregate their data which is difficult due to privacy-related and logistical concerns.\nThisstudyanalysestheimpactofthenetworktopologyoffederatedlearningontheperformance\nof the predictive model for the case of STLF. Furthermore, since not all clients benefit from fed-\nerated learning, the study of predictive models for distinguishing benefitting from non-benefitting\nclients is also carried out.\nThe ring topology results in the best performance in terms of the n

In [8]:
info = ''
j = 0
for i in pages[0:50]:
    info = info + i.page_content
    texts = text_splitter.create_documents([i.page_content])
    documents = [i.page_content for i in texts]
    metadatas = [i.metadata for i in texts]
    ids = [str(j+i) for i in range(len(documents))]
    j = j + len(documents)
    print(ids)
    collection.add(
    documents = documents,
    metadatas = metadatas,
    ids = ids
    )

['0']
['1', '2']
['3']
['4']
['5']
['6']
['7']
['8']
['9']
['10', '11']
['12']
['13']
['14']
['15', '16', '17']
['18']
['19']
['20']
['21', '22', '23']
['24', '25', '26', '27']
['28']
['29']
['30', '31']
['32', '33', '34']
['35', '36', '37']
['38', '39', '40', '41']
['42', '43', '44', '45']
['46', '47', '48']
['49', '50']
['51', '52']
['53', '54', '55']
['56', '57', '58', '59']
['60', '61', '62']
['63', '64', '65']
['66', '67', '68']
['69']
['70', '71']
['72', '73', '74']
['75', '76']
['77', '78']
['79', '80']
['81', '82']
['83', '84']
['85', '86', '87']
['88', '89', '90']
['91', '92', '93']
['94', '95', '96', '97']
['98', '99', '100']
['101', '102']
['103', '104', '105']
['106', '107', '108']


In [9]:
question = "Who is my supervisor?"
results = collection.query(query_texts = [question], n_results = 5)

In [10]:
results

{'ids': [['8', '20', '69', '18', '4']],
 'distances': [[1.407895565032959,
   1.641662836074829,
   1.6523849964141846,
   1.6579170227050781,
   1.6685047149658203]],
 'metadatas': [[{'start_index': 0},
   {'start_index': 0},
   {'start_index': 0},
   {'start_index': 0},
   {'start_index': 0}]],
 'embeddings': None,
 'documents': [['Acknowledgments\nI would firstly like to thank my supervisor Dr Fei Teng for encouraging thought provoking discus-\nsions and providing incomparbale guidance throughout this thesis. I am also very thankful for Mr\nXu’s for valuable inputs and insights on this thesis.\nFinally, I am very grateful to my friends and family for continuously supporting me throughout\nthis degree.',
   'xvi',
   '218 CHAPTER 2. LITERATURE REVIEW',
   'xiv',
   'Declaration of Originality\nI hereby declare that the work presented in this thesis is my own unless otherwise stated. To the\nbest of my knowledge the work is original and ideas developed in collaboration with others hav

In [11]:
len(results['documents'][0])

5

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

results = collection.query(query_texts = [question], n_results = 2)

template = """The following piece of text is given: {text}. Please answer any following questions ONLY using THIS piece of text in a brief manner."""
human_template = "{question}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() 
context = ''
for i in range(len(results['documents'])):
    context = context + results['documents'][0][i] + '\n'
chain.invoke({"text": context, "question": question})


AIMessage(content='Your supervisor is Dr Fei Teng.')

In [13]:
# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1);
# chat_model = ChatOpenAI();
# llm.predict("hi! how are you?")

In [14]:
# from langchain.prompts.chat import ChatPromptTemplate

# template = "You are a helpful assistant that translates {input_language} to {output_language}."
# human_template = "{text}"

# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template),
#     ("human", human_template),
# ])

# x = chat_prompt.format_messages(input_language="English", output_language="French", text="Okay")